___

# Tidy Data in python

___



In [1]:
import pandas as pd
import datetime
import glob
import re

In [2]:
baby2014 = pd.read_csv('2014-baby-names-illinois.csv')
baby2015 = pd.read_csv('2015-baby-names-illinois.csv')
billboard = pd.read_csv('billboard.csv',encoding='mac_latin2')
pew = pd.read_csv('pew.txt', sep="\t")
tb = pd.read_csv('tb.csv')
weather = pd.read_csv('weather.txt', sep="\t")

# Pew

In [3]:
#Original
pew.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [4]:
tidypew = pd.melt(pew,["religion"], var_name="income", value_name="freq")
tidypew = tidypew.sort_values(by=["religion"])
tidypew.head()

,religion,income,freq
0,Agnostic,<$10k,27
144,Agnostic,>150k,84
162,Agnostic,Don't know/refused,96
126,Agnostic,$100-150k,109
36,Agnostic,$20-30k,60


# Billboard

In [5]:
#Original
billboard.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Melt
ids = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
billboardtidy = pd.melt(frame=billboard,id_vars=ids, var_name="week", value_name="rank")

In [7]:
billboardtidy.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


In [8]:
# Drop NaN
billboardtidy = billboardtidy.dropna()
# Correct format 
billboardtidy['week'] = billboardtidy['week'].str.extract('(\d+)', expand=False).astype(int)
billboardtidy['rank'] = billboardtidy['rank'].astype(int)

In [9]:
billboardtidy.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57


In [10]:
# Date col
billboardtidy['date'] = pd.to_datetime(billboardtidy['date.entered']) + pd.to_timedelta(billboardtidy['week'], unit='w') - pd.DateOffset(weeks=1)

In [11]:
billboardtidy.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78,2000-09-23
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71,1999-10-23
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57,2000-08-05


In [12]:
#Select variables and do ascending sort by the following list.
billboardtidy = billboardtidy[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]
billboardtidy = billboardtidy.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])
billboardtidy.head(10)

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25
1831,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,6,94,2000-04-01
2148,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,7,99,2000-04-08
287,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1,91,2000-09-02
604,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,2,87,2000-09-09
921,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,3,92,2000-09-16


## Multiple types in one table

In [13]:
#Drop duplicates
singles = billboardtidy[["year", "artist.inverted", "track", "time", "genre"]].drop_duplicates()
#Reset index and assign id's
singles = singles.reset_index(drop=True)
singles["id"] = singles.index
singles.head(10)

,year,artist.inverted,track,time,genre,id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,0
1,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1
2,2000,3 Doors Down,Kryptonite,3:53,Rock,2
3,2000,3 Doors Down,Loser,4:24,Rock,3
4,2000,504 Boyz,Wobble Wobble,3:35,Rap,4
5,2000,98°,Give Me Just One Night (Una Noche),3:24,Rock,5
6,2000,A*Teens,Dancing Queen,3:44,Pop,6
7,2000,Aaliyah,I Don't Wanna,4:15,Rock,7
8,2000,Aaliyah,Try Again,4:03,Rock,8
9,2000,"Adams, Yolanda",Open My Heart,5:30,Gospel,9


In [14]:
#Merge
ranking = pd.merge(billboardtidy, singles, on=["year", "artist.inverted", "track", "time", "genre"])
ranking.head(10)

,year,artist.inverted,track,time,genre,week,rank,date,id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26,0
1,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04,0
2,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11,0
3,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18,0
4,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25,0
5,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,6,94,2000-04-01,0
6,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,7,99,2000-04-08,0
7,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1,91,2000-09-02,1
8,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,2,87,2000-09-09,1
9,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,3,92,2000-09-16,1


In [15]:
#Nav trough ID.
ranking = ranking[["id", "date", "rank"]]
ranking.head(10)

,id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87
5,0,2000-04-01,94
6,0,2000-04-08,99
7,1,2000-09-02,91
8,1,2000-09-09,87
9,1,2000-09-16,92


# Tubercolosis
##  Multiple variables stored in one column

In [16]:
tb.head() 

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
tb.columns = tb.columns.str.replace('new_sp_','')
tb.rename(columns = {'iso2' : 'country'}, inplace=True) 
tb = tb[tb['year'] == 2000]
tb.drop(['new_sp', 'm04', 'm514', 'f04', 'f514'], axis=1, inplace=True)
tb.iloc[:,:11].head(10) #Cleaned up

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
10,AD,2000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
36,AE,2000,2.0,4.0,4.0,6.0,5.0,12.0,10.0,NaN,3.0
60,AF,2000,52.0,228.0,183.0,149.0,129.0,94.0,80.0,NaN,93.0
87,AG,2000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0
136,AL,2000,2.0,19.0,21.0,14.0,24.0,19.0,16.0,NaN,3.0
165,AM,2000,2.0,152.0,130.0,131.0,63.0,26.0,21.0,NaN,1.0
178,AN,2000,0.0,0.0,1.0,2.0,0.0,0.0,0.0,NaN,0.0
207,AO,2000,186.0,999.0,1003.0,912.0,482.0,312.0,194.0,NaN,247.0
236,AR,2000,97.0,278.0,594.0,402.0,419.0,368.0,330.0,NaN,121.0
265,AS,2000,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN


In [18]:
tb = pd.melt(tb, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")

In [19]:
tb.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


In [20]:
# Extract sex, lower age and upper age ranges.
temptb = tb["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)

In [21]:
temptb.head()

,0,1,2
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14


In [22]:
# give a name to cols
temptb.columns = ["sex", "age_lower", "age_upper"]
# Create age range string
temptb["age"] = temptb["age_lower"] + "-" + temptb["age_upper"]

In [23]:
temptb.head()

,sex,age_lower,age_upper,age
0,m,0,14,0-14
1,m,0,14,0-14
2,m,0,14,0-14
3,m,0,14,0-14
4,m,0,14,0-14


In [24]:
# Merge them
tb = pd.concat([tb, temptb], axis=1)

In [25]:
tb.head()

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14


In [26]:
# Clean and sort.
tb = tb.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
tb = tb.dropna()
tb = tb.sort_values(ascending=True,by=["country", "year", "sex", "age"])
tb.head(10)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
201,AD,2000,0.0,m,15-24
402,AD,2000,1.0,m,25-34
603,AD,2000,0.0,m,35-44
804,AD,2000,0.0,m,45-54
1005,AD,2000,0.0,m,55-64
1609,AE,2000,3.0,f,0-14
1810,AE,2000,16.0,f,15-24
2011,AE,2000,1.0,f,25-34
2212,AE,2000,3.0,f,35-44


# Weather
## Variables are stored in rows and cols.

In [27]:
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX000017004,2010,1,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.0,NaN
1,MX000017004,2010,1,TMIN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN
2,MX000017004,2010,2,TMAX,NaN,273.0,241.0,NaN,NaN,NaN,...,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX000017004,2010,2,TMIN,NaN,144.0,144.0,NaN,NaN,NaN,...,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX000017004,2010,3,TMAX,NaN,NaN,NaN,NaN,321.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
weather = pd.melt(weather, id_vars=["id", "year","month","element"], var_name="day")
weather.dropna(inplace=True)
weather = weather.reset_index(drop=True)
weather.head()

,id,year,month,element,day,value
0,MX000017004,2010,12,TMAX,d1,299.0
1,MX000017004,2010,12,TMIN,d1,138.0
2,MX000017004,2010,2,TMAX,d2,273.0
3,MX000017004,2010,2,TMIN,d2,144.0
4,MX000017004,2010,11,TMAX,d2,313.0


In [29]:
def formater(row):    
    return "%d-%02d-%02d" % (row['year'], row['month'], int(row['day'][1:]))

weather['date'] = weather.apply(formater,axis=1)
weather = weather[['id', 'element','value','date']]
weather.head()

,id,element,value,date
0,MX000017004,TMAX,299.0,2010-12-01
1,MX000017004,TMIN,138.0,2010-12-01
2,MX000017004,TMAX,273.0,2010-02-02
3,MX000017004,TMIN,144.0,2010-02-02
4,MX000017004,TMAX,313.0,2010-11-02


In [30]:
weather = weather.groupby('id').apply(pd.DataFrame.pivot,
                                  index='date',
                                  columns='element',
                                  values='value')
weather.reset_index(inplace=True)

In [31]:
weather.head()

element,id,date,TMAX,TMIN
0,MX000017004,2010-01-30,278.0,145.0
1,MX000017004,2010-02-02,273.0,144.0
2,MX000017004,2010-02-03,241.0,144.0
3,MX000017004,2010-02-11,297.0,134.0
4,MX000017004,2010-02-23,299.0,107.0


# Baby Names 
## One type multiple tables

In [32]:
baby2014.head()

,rank,name,frequency,sex
0,1,Noah,837,Male
1,2,Alexander,747,Male
2,3,William,687,Male
3,4,Michael,680,Male
4,5,Liam,670,Male


In [33]:
baby2015.head()

,rank,name,frequency,sex
0,1,Noah,863,Male
1,2,Liam,709,Male
2,3,Alexander,703,Male
3,4,Jacob,650,Male
4,5,William,618,Male


In [34]:
def year(string):
    match = re.match("(\d{4})", string) 
    if match != None:
        return int(match.group(1))
    
csvfiles = glob.glob("201*-baby-names-illinois.csv")

In [35]:
dflist = []
for f in csvfiles:
    df = pd.read_csv(f, index_col = None, header = 0)
    df.columns = map(str.lower, df.columns)
    df["year"] = year(f)
    dflist.append(df)

In [36]:
babynames = pd.concat(dflist)
babynames.head(5)

,rank,name,frequency,sex,year
0,1,Noah,837,Male,2014
1,2,Alexander,747,Male,2014
2,3,William,687,Male,2014
3,4,Michael,680,Male,2014
4,5,Liam,670,Male,2014


## Thats it! 